In [1]:
from datetime import datetime, timedelta

class Task:
    def __init__(self, task_id, name, priority, duration, required_skills, due_date):
        self.task_id = task_id
        self.name = name
        self.priority = priority
        self.duration = duration  # in hours
        self.required_skills = required_skills
        self.due_date = due_date

class Worker:
    def __init__(self, worker_id, name, skills, availability):
        self.worker_id = worker_id
        self.name = name
        self.skills = skills
        self.availability = availability  # List of tuples [(start_time, end_time)]

class Resource:
    def __init__(self, resource_id, name, capacity):
        self.resource_id = resource_id
        self.name = name
        self.capacity = capacity
